# Neural ODE Net

## Background

In [2]:
from IPython.display import Image 
url='https://m0nads.files.wordpress.com/2019/06/rect1494.png'
Image(url=url)

- Traditional deep learning models use discrete layers and fixed architectures
    
    e.g., **ResNet**: uses skip connections to mitigate vanishing and exploding gradients in deep neural networks.


- Neural ODE Net is a new type of **infinite deep** NN based on ODE that provide a continuous-time alternative for modeling complex systems

    2-4 x depth of RedNet


- **Dynamic system**: A system whose behavior changes over time based on a set of rules, often modeled using ODEs.

- **Ordinary Differential Equation (ODE)**: A mathematical equation that describes the relationship between a function $z$ and its derivatives $f$, often used to model dynamic systems.

    $$
    \frac{dz(t)}{dt} = f(z(t), t)\\[1em]
    \text{Initial condition:\ } z(t_0) = z_0
    $$

- Initial Value Problem:

    Given an ODE $\frac{d\mathbf{z}(t)}{dt} = f(\mathbf{z}(t), t; \theta)$ with initial condition $\mathbf{z}(t_0) = \mathbf{z}_0$,

    the goal is to find the function $\mathbf{z}(t)$ that satisfies the ODE and the initial condition.

- **Euler Integrators**: A family of numerical methods for solving ODEs by approximating the solution at discrete time steps $t_0, t_1, t_2, \dots$ with small Step size: $h$, The update rule is: 

    $$z(t+h) = z(t) + hf(z, t)$$



## Algorithm for training the latent ODE model

To obtain the latent representation $\mathbf{z}_{t_0}$ (initial state of the system at time $t_0$), we traverse the sequence using RNN and obtain parameters $\phi$ of variational distribution $q(\mathbf{z}_{t_0}|\{\mathbf{x}_{t_i}, t_i\}_i, \phi)$. 


In [1]:
from IPython.display import Image 
url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*LRtGTST73BgRE33BgXS2AA.png'
Image(url=url)

The algorithm follows a standard VAE algorithm with an RNN variational posterior and an ODESolve model:

1. Run an RNN encoder through the time series and infer the parameters for a posterior over $\mathbf{z}_{t_0}$: 

$$q(\mathbf{z}_{t_0}|\{\mathbf{x}_{t_i}, t_i\}_i, \phi) = \mathcal{N}(\mathbf{z}_{t_0}|\mu_{\mathbf{z}_{t_0}}, \sigma_{\mathbf{z}_0})$$

- $\mu_{\mathbf{z}_{t_0}}, \sigma_{\mathbf{z}_0}$ are mean and standard deviation comes from the hidden state $\mathbf{h}_{t_0}, ..., \mathbf{h}_{t_N}$ of $\text{RNN}(\{\mathbf{x}_{t_i}, t_i\}_i, \phi)$

- $\{\mathbf{x}_{t_i}, t_i\}_i$: input sequence with observations $\mathbf{x}_{t_i}$ and corresponding times $t_i$
   
2. Sample latent representation from posterior distribution

$$
\mathbf{z}_{t_0} \sim q(\mathbf{z}_{t_0}|\{\mathbf{x}_{t_i}, t_i\}_i)
$$

3. Obtain $\mathbf{z}_{t_1}, \mathbf{z}_{t_2},..., \mathbf{z}_{t_M}$ by solving ODE 

$$\text{ODESolve}(\mathbf{z}_{t_0}, f, \theta, t_0,...,t_M)$$

- $f$: the neural network representing the ODE dynamics, parameterized by $\theta$. defining the gradient $d\mathbf{z}/dt$ as a function of $\mathbf{z}$

- $\text{ODESolve}$ is a numerical solver that approximates the solution of the ODE. e.g., Runge-Kutta method.


4. Maximize evidence lower bound (ELBO), a lower bound on the log-likelihood of the data

$$
\text{ELBO} = \frac{1}{M}\sum_{i=1}^M \log p(\mathbf{x}_{t_i}|\mathbf{z}_{t_i}, \theta_x) + \log p(\mathbf{z}_{t_0}) - \log q(\mathbf{z}_{t_0}|\{\mathbf{x}_{t_i}, t_i\}_i, \phi)
$$

- $p(\mathbf{x}_{t_i}|\mathbf{z}_{t_i}, \theta_x)$: likelihood of the observed data $\mathbf{x}_{t_i}$ given the latent variable $\mathbf{z}_{t_i}$ and the model parameters $\theta_x$

- $p(\mathbf{z}_{t_0}) = \mathcal{N}(0, 1)$: prior distribution over latent variable $\mathbf{z}_{t_0}$, which is standard normal

- $q(\mathbf{z}_{t_0}|{\mathbf{x}_{t_i}, t_i}_i, \phi)$: posterior  distribution over latent variable $\mathbf{z}_{t_0}$ given the input sequence ${\mathbf{x}_{t_i}, t_i}_i^M$ and variational parameters $\phi$

- the sum is taken over all time steps $t_1, t_2, ..., t_M$

- $M$: length of the time series and determines the granularity of the latent representation.



5. Compute the loss: 

   $$
   L(\mathbf{z}(t_1)) = L(\mathbf{z}(t_0) + \int_{t_0}^{t_1} f(\mathbf{z}(t), t, \theta)dt) = L(\text{ODESolve}(\mathbf{z}(t_0), f, t_0, t_1, \theta))
   $$

- $L$ is negative log likelihood loss function.
- $\mathbf{z}(t_0)$ is the initial state of the system at time $t_0$.
- $\mathbf{z}(t_1)$ is the predicted state of the system at time $t_1$.


6. Backward pass: Calculate gradients of loss w.r.t model parameters $\theta$ and initial state $\mathbf{z}_0$ using the **adjoint sensitivity** method 

   - Define the adjoint state as derivative of loss w.r.t hidden state $\mathbf{z}(t)$ at intermediate timepoint $t$
   
      $$\mathbf{a}(t) = \frac{dL}{d\mathbf{z}(t)}$$

   - Solve the adjoint ODE 
   
      $$\frac{d\mathbf{a}(t)}{dt} = -\mathbf{a}(t)^T \frac{df(\mathbf{z}(t), t; \theta)}{d\mathbf{z}}
      $$ 
      
      backward in time with terminal condition $\mathbf{a}(t_1) = \frac{dL}{d\mathbf{z}(t_1)}$.

   - Compute the gradients: 
   
   $$
   \frac{dL}{d\theta} = \int_{t_0}^{t_1} \mathbf{a}(t)^T \frac{df(\mathbf{z}(t), t; \theta)}{d\theta} dt\\[1em]
   \frac{dL}{d\mathbf{z}_{t_0}} = \mathbf{a}(t_0)
   $$


7. Update the parameters: Update the model parameters $\theta$ using an optimization algorithm (e.g., gradient descent) and the computed gradients.

In [3]:
url='https://www.researchgate.net/profile/Xiang-Xie/publication/337539890/figure/fig2/AS:872753787846656@1585092126559/depicts-this-unrolled-architecture-of-neural-ODE-based-encoder-decoder-modules-used-for.jpg'
Image(url=url)

## advantage

Memory-efficient: Neural ODEs use **adjoint sensitivity** method to compute gradient, which don't need directly backpropagating through the ODE solver.

Adaptive computation: The ODE solver can adapt its computation to the complexity of the problem, allocating more resources when needed and reducing computations for simpler cases.

Continuous-time models: Unlike recurrent neural networks, which require discretizing observation and emission intervals, continuously-defined dynamics can naturally incorporate data
which arrives at arbitrary times, which can provide better representations for certain applications, such as time series analysis, physical simulations, or biological systems.


Scalable and invertible normalizing flows: change of variables formula becomes easier to compute. we construct a new class of invertible density models that avoids the single-unit bottleneck of normalizing flows, and can be trained directly by maximum likelihood

# normalizing flow

A normalizing flow is a **generative model** that transforms a simple base distribution $Q$ (e.g., Gaussian) into a more complex distribution $P$ by applying a series of invertible and differentiable transformations $f$

## deep normalizing flow

Training Algorithm:

1. Sample a batch of points $z_0$ from the base distribution $Q$.

2. Apply the series of invertible and differentiable transformations to obtain $z(t)$.

    $$z(t_1) = f(z(t_0)) \Rightarrow z(t_N) = f_N \circ f_{N-1} \circ \cdots \circ f_1 (z(t_0))$$

3. Compute log-likelihood of the transformed points $z(t)$ using **change of variables formula**.

  $$
  \log p(z(t_1)) = \log p(z(t_0)) - \log \left| \det \frac{\partial f}{\partial z(t_0)} \right| \Rightarrow
  \log p(z(t_N)) = \log p(z(t_0)) - \sum_{n=1}^N\log \left| \det \frac{\partial f_n}{\partial z(t_{n-1})} \right|
  $$

4. Maximize log-likelihood w.r.t the parameters of the transformations using an optimization algorithm

$$
\max_Q \mathbb{E}_{P}\log Q
$$

## continous normalizing flow

Training Algorithm:

1. Sample a batch of points $z_0$ from the base distribution $Q$.

2. Apply the series of invertible and differentiable transformations to obtain $z(t)$.

    $$
    z(t_N) = F(z(t_0)) = \int_{t_0}^{t_N} f(z(t_N), t)dt
    $$

3. Compute log-likelihood of the transformed points $z(t)$ using **change of variables formula**.

  $$ 
  \log p(z(t_N)) = \log p(z(t_0)) - \int_{t_0}^{t_N} \text{Trace}\left( \frac{\partial f}{\partial z(t)}\right)dt
  $$

4. Maximize log-likelihood w.r.t the parameters of the transformations using an optimization algorithm

$$
\max_Q \mathbb{E}_{P}\log Q
$$